Import the libraries needed

In [1]:
import pandas as pd
#from pandas.compat import StringIO
from pandas.tseries.offsets import DateOffset

from datetime import datetime
import psutil


import numpy as np
import os
import pyodbc
pyodbc.drivers()

import csv

from sklearn.preprocessing import StandardScaler
import seaborn as sns

pd.options.display.max_rows = 1000

# For the Yahoo Finance api
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override() 


C:\Users\m033\Anaconda3\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.16) or chardet (3.0.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn(


### Database connections and SQL to retrieve the various data needed.

In [2]:
SQL_server = 'WINPRDAI1572\CLESQLDW'
Reporting_database = 'CLE_Reporting'
Reference_Data_database = 'CLE_Reference_Data'

pd.set_option('max_row', None)
#pd.options.display.max_rows = 1000

# Set a value for the home folder.
home_folder = "."

# Set values for the various paths.
input_path = home_folder + "\data"

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

2024-07-19 19:08:15.278437
svmem(total=16756752384, available=3544735744, percent=78.8, used=13212016640, free=3544735744)


# 1. Get supplementary Asset features.

  -  Identified in Lit Review.

In [3]:
conn = pyodbc.connect(
         'DRIVER={ODBC Driver 17 for SQL Server};'
         'SERVER='+SQL_server+';'
         'DATABASE='+Reporting_database+';'
         'Trusted_Connection=yes;')

asset_query = f"""SELECT pol.Policy_Number         AS ContractNumber
                        ,pol.Pricing_Variant_Id           AS PricingVariant
                        ,CASE WHEN f.Asset_Type_Code = 'UW' THEN 1
                             ELSE 0 
                        END as Has_UWP
                 FROM CLT_Policy pol
                 INNER JOIN CLE_Reporting.dbo.CLT_Product prd ON prd.Product_Id = pol.Product_Id
                 INNER JOIN CLT_Policy_Fund_Values fv ON fv.Policy_Number = pol.Policy_Number and  fv.Valuation_Date = '2024-07-03' 
                 INNER JOIN CLE_Reporting.[user].CLV_Fund F ON fv.Fund_Name = f.Fund_Name AND f.Asset_Type_Code = 'UW'
                 WHERE prd.Product_Code IN ('FKP_2006V1','FKP_2006V2','FKP_2008V1','FKP_2010V1','FKP_2012V1','FKP_2017V1',
                                            'GNPP_2016V1','GNPP_2017V1','GNPP_2022V1','GNPP_2023V1',
                                            'GNRP_2008V1','GNRP_2010V1','GNRP_2012V1','GNRP_2017V1','GNRP_2022V1',
                                            'XRPP_2005V1','XRPP_2008V1','XRPP_2010V1','XRPP_2012V1','XRPP_2015V1')
                 GROUP BY pol.Policy_Number  ,f.Asset_Type_Code ,Pricing_Variant_Id                          
                 ORDER BY pol.Policy_Number;"""

df_asset = pd.read_sql(asset_query, conn)

# Types and row count
# print stats.
print(len(df_asset.index))
now = datetime.now()
print(now)
print('==========================')

print(df_asset.dtypes)
df_asset.head(5)

100309
2024-07-19 19:09:18.950102
ContractNumber    object
PricingVariant     int64
Has_UWP            int64
dtype: object


,ContractNumber,PricingVariant,Has_UWP
0,6215812A,4015,1
1,6216004M,4015,1
2,6216180E,4015,1
3,6216746P,4015,1
4,6217141W,4015,1


## Drop obsolette or repeated features.

## Check for nulls

In [4]:
df_asset.isna().any()

ContractNumber    False
PricingVariant    False
Has_UWP           False
dtype: bool

## Check the distributions of values for each column.

In [5]:
print('PricingVariant : ',df_asset['PricingVariant'].value_counts())
print('Has_UWP        : ',df_asset['Has_UWP'].value_counts())


PricingVariant :  6       94518
4        1675
5        1150
4001      773
4015      727
4004      599
4017      433
4002      179
4005      142
4003      103
4016        6
4036        2
4045        1
4044        1
Name: PricingVariant, dtype: int64
Has_UWP        :  1    100309
Name: Has_UWP, dtype: int64


## Has_UWP is showing no diatribution across multiple values and is dropped.

## Drop all non predictive features and write to Excel

In [6]:
df_asset = df_asset.drop(['Has_UWP'],axis=1)

# Write out to a comma separated values file.

# giving directory name
filename = input_path + '\Asset_data.csv'
df_asset.to_csv(filename, encoding='utf-8', index=False)        
df_asset.head(10)

,ContractNumber,PricingVariant
0,6215812A,4015
1,6216004M,4015
2,6216180E,4015
3,6216746P,4015
4,6217141W,4015
5,6217672P,4015
6,6218494M,4015
7,6218651L,4015
8,6218789F,4015
9,6218794A,4015


In [7]:
%who_ls DataFrame 

['df_asset']

In [8]:
del df_asset

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

2024-07-19 19:09:19.425092
svmem(total=16756752384, available=2924802048, percent=82.5, used=13831950336, free=2924802048)
